In [ ]:
# 计算目录下的json文件总数
import os
import json

def count_json_objects(directory):
    total_objects = 0
    # 检查目录是否存在
    if not os.path.isdir(directory):
        print(f"错误：目录 '{directory}' 不存在。")
        return -1

    # 遍历目录中的所有文件
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # 如果JSON文件内容是一个列表，计算其元素的个数
                    if isinstance(data, list):
                        total_objects += len(data)
                    # 如果JSON文件内容是一个字典，则它本身是一个对象
                    elif isinstance(data, dict):
                        total_objects += 1
            except json.JSONDecodeError as e:
                print(f"警告：文件 '{filepath}' 不是有效的 JSON 格式。错误：{e}")
            except Exception as e:
                print(f"处理文件 '{filepath}' 时发生错误：{e}")
    return total_objects

# 定义目标目录
directory_path = "/home/hsh/Y3S1/gao1119/tw_no_empty" 

In [1]:
# 计算单个json文件中的对象数量
import json

# 文件路径
# file_path = "/mnt/disk60T/dataset/Culture/Museum/All/zhejiang_integrated(1).json" # final 5609
file_path = "/mnt/disk60T/dataset/Culture/Museum/Final_version/Final_Zhejiang.json" 

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

if isinstance(data, list):
    count = len(data)
elif isinstance(data, dict):
    count = len(data)
else:
    count = 0

print(f"{file_path}中JSON对象数量：{count}")

/mnt/disk60T/dataset/Culture/Museum/Final_version/Final_Zhejiang.json中JSON对象数量：5750


In [ ]:
import json

# ==============================
# 需要被统计总数的json文件路径
# ==============================
json_paths = [
    "/mnt/disk60T/dataset/Culture/Museum/Final_version/Final_gugong.json",
    "/mnt/disk60T/dataset/Culture/Museum/Final_version/Final_Taiwan.json",
    "/mnt/disk60T/dataset/Culture/Museum/Final_version/Final_Zhejiang.json",
]

def count_images(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    total = 0
    for obj in data:
        # 优先使用 "images"，若没有则尝试 "图片"
        imgs = obj.get("images") or obj.get("图片") or []
        if isinstance(imgs, list):
            total += len(imgs)
    return total

results = {}
grand_total = 0

for path in json_paths:
    count = count_images(path)
    results[path] = count
    grand_total += count

for path, count in results.items():
    print(f"{path} 中共有 {count} 张图片")

print(f"\n✅ JSON 文件合计共有 {grand_total} 张图片")

In [ ]:
# 合并json成大json文件
import json
from pathlib import Path

PATHS = [
    Path("/home/hsh/Y3S1/data/tw_blip1.json"),
    Path("/home/hsh/data/bj_blip1.json"),
    Path("/home/hsh/data/zj_blip1.json"),
]

OUTPUT_PATH = Path("/home/hsh/data/combined_blip1.json")

def load_json(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

def combine_json(paths):
    combined = []
    for path in paths:
        data = load_json(path)
        if isinstance(data, list):
            combined.extend(data)
        else:
            combined.append(data)
    return combined

def save_json(data, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    result = combine_json(PATHS)
    save_json(result, OUTPUT_PATH)
    print(f"Combined {len(PATHS)} files into '{OUTPUT_PATH}'.")

In [ ]:
# check blip1 dim
import json

def check_json_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not isinstance(data, list):
            print("异常：最外层不是一个列表。")
            return

        for i, obj in enumerate(data):
            if not isinstance(obj, dict):
                print(f"异常：索引为 {i} 的元素不是一个字典。")
                continue

            # 检查 jianjie_embedding_blip1 (保持不变，768维)
            if "jianjie_embedding_blip1" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'jianjie_embedding_blip1' 字段。")
            elif not isinstance(obj["jianjie_embedding_blip1"], list) or len(obj["jianjie_embedding_blip1"]) != 768:
                print(f"异常：索引为 {i} 的对象中 'jianjie_embedding_blip1' 不是768维的列表。")

            # 检查 structured_embedding_blip1 (保持不变，768维)
            if "structured_embedding_blip1" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'structured_embedding_blip1' 字段。")
            elif not isinstance(obj["structured_embedding_blip1"], list) or len(obj["structured_embedding_blip1"]) != 768:
                print(f"异常：索引为 {i} 的对象中 'structured_embedding_blip1' 不是768维的列表。")

            # 检查 image_embeddings_blip1 (修改为检查其字典中每个值的维度，1024维)
            if "image_embeddings_blip1" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'image_embeddings_blip1' 字段。")
            elif not isinstance(obj["image_embeddings_blip1"], dict):
                print(f"异常：索引为 {i} 的对象中 'image_embeddings_blip1' 不是一个字典。")
            else:
                for key, embedding_list in obj["image_embeddings_blip1"].items():
                    if not isinstance(embedding_list, list) or len(embedding_list) != 1024:
                        print(f"异常：索引为 {i} 的对象中 'image_embeddings_blip1' 字典键 '{key}' 对应的值不是1024维的列表。")

        print("检查完成。如果上面没有异常信息，则所有对象都符合要求。")

    except FileNotFoundError:
        print(f"错误：文件 '{file_path}' 未找到。")
    except json.JSONDecodeError:
        print(f"错误：文件 '{file_path}' 不是一个有效的JSON文件。")
    except Exception as e:
        print(f"发生未知错误: {e}")

# 调用函数来检查你的文件
file_path = "/home/hsh/Y3S1/data/tw_blip1.json"
check_json_data(file_path)

In [ ]:
# check blip2 dim
import json

def check_json_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not isinstance(data, list):
            print("异常：最外层不是一个列表。")
            return

        for i, obj in enumerate(data):
            if not isinstance(obj, dict):
                print(f"异常：索引为 {i} 的元素不是一个字典。")
                continue

            # 检查 jianjie_embedding_blip2
            if "jianjie_embedding_blip2" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'jianjie_embedding_blip2' 字段。")
            elif not isinstance(obj["jianjie_embedding_blip2"], list):
                print(f"异常：索引为 {i} 的对象中 'jianjie_embedding_blip2' 不是一个列表。")
            elif len(obj["jianjie_embedding_blip2"]) != 1 or not isinstance(obj["jianjie_embedding_blip2"][0], list):
                print(f"异常：索引为 {i} 的对象中 'jianjie_embedding_blip2' 格式不正确，期望是 [[...]] 形式的嵌套列表。")
            else:
                actual_embedding = obj["jianjie_embedding_blip2"][0] # 取出内层列表
                if len(actual_embedding) != 768:
                    print(f"异常：索引为 {i} 的对象中 'jianjie_embedding_blip2' 期望是768维，实际是 {len(actual_embedding)} 维。")


            # 检查 structured_embedding_blip2
            if "structured_embedding_blip2" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'structured_embedding_blip2' 字段。")
            elif not isinstance(obj["structured_embedding_blip2"], list):
                print(f"异常：索引为 {i} 的对象中 'structured_embedding_blip2' 不是一个列表。")
            elif len(obj["structured_embedding_blip2"]) != 1 or not isinstance(obj["structured_embedding_blip2"][0], list):
                print(f"异常：索引为 {i} 的对象中 'structured_embedding_blip2' 格式不正确，期望是 [[...]] 形式的嵌套列表。")
            else:
                actual_embedding = obj["structured_embedding_blip2"][0] # 取出内层列表
                if len(actual_embedding) != 768:
                    print(f"异常：索引为 {i} 的对象中 'structured_embedding_blip2' 期望是768维，实际是 {len(actual_embedding)} 维。")


            # 检查 image_embeddings_blip2
            if "image_embeddings_blip2" not in obj:
                print(f"异常：索引为 {i} 的对象缺少 'image_embeddings_blip2' 字段。")
            elif not isinstance(obj["image_embeddings_blip2"], dict):
                print(f"异常：索引为 {i} 的对象中 'image_embeddings_blip2' 不是一个字典。")
            else:
                for key, embedding_list_outer in obj["image_embeddings_blip2"].items():
                    if not isinstance(embedding_list_outer, list) or len(embedding_list_outer) != 1 or not isinstance(embedding_list_outer[0], list):
                        print(f"异常：索引为 {i} 的对象中 'image_embeddings_blip2' 字典键 '{key}' 对应的值格式不正确，期望是 [[...]] 形式的嵌套列表。")
                    else:
                        actual_embedding = embedding_list_outer[0] # 取出内层列表
                        if len(actual_embedding) != 768: # 这里的期望维度是768，如果实际是1024，请修改为1024
                            print(f"异常：索引为 {i} 的对象中 'image_embeddings_blip2' 字典键 '{key}' 对应的值期望是768维，实际是 {len(actual_embedding)} 维。")

        print("检查完成。如果上面没有异常信息，则所有对象都符合要求。")

    except FileNotFoundError:
        print(f"错误：文件 '{file_path}' 未找到。")
    except json.JSONDecodeError:
        print(f"错误：文件 '{file_path}' 不是一个有效的JSON文件。")
    except Exception as e:
        print(f"发生未知错误: {e}")

# 调用函数来检查你的文件
file_path = "/home/hsh/Y3S1/data/tw_blip2.json"
check_json_data(file_path)